In [1]:
from __future__ import print_function
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from skimage.io import imsave
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from IPython.display import Image
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import glob
from loader import ImageFolder
nz = 100
nb_classes = 18
ngf = 64
ndf = 64
nc = 3
class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz+nb_classes , ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        return nn.parallel.data_parallel(self.main, input, gpu_ids)

In [2]:
import sys
def compute_objectness(v):
        marginal = v.mean(dim=0)
        marginal = marginal.repeat(v.size(0), 1)
        score = v * torch.log(v / (marginal))
        score = score.sum(dim=1).mean()
        return score
    
sys.path.append('pytorch_pretrained')
clf = torch.load('pytorch_pretrained/clf-4cf42cbb-3c69-4d3f-86e1-c050330a7c7c.th')
clf = clf.cuda()

clf_mean = np.array([0.485, 0.456, 0.406], dtype='float32')
clf_mean = clf_mean[np.newaxis, :, np.newaxis, np.newaxis]
clf_mean = torch.from_numpy(clf_mean)
clf_mean = Variable(clf_mean)
clf_mean = clf_mean.cuda()
clf_std = np.array([0.229, 0.224, 0.225], dtype='float32')
clf_std = clf_std[np.newaxis, :, np.newaxis, np.newaxis]
clf_std = torch.from_numpy(clf_std)
clf_std = Variable(clf_std)
clf_std = clf_std.cuda()

def norm(x):
    x = (x + 1) / 2.
    x += 1
    x /= 2.
    x -= clf_mean.repeat(x.size(0), 1, x.size(2), x.size(3))
    x /= clf_std.repeat(x.size(0), 1, x.size(2), x.size(3))
    return x

ImportError: No module named 'batch_classifier'

In [ ]:
G = _netG(ngpu=1)
G.load_state_dict(torch.load('samples_128_cond_3/netG_epoch_7.pth'))

In [ ]:

# for mode random and random_same_for_each_class
nb_rows = 10
nb_cols = nb_classes

# for mode interp_same_for_each_class
nb_rows = nz
nb_cols = 20
CL = 6

#mode = 'random'
mode = 'interp_same_for_each_class'
if mode == 'random':
    fixed_z = torch.randn   (nb_rows, nb_cols,        nz, 1, 1)
    fixed_z = fixed_z.view(nb_rows * nb_cols, nz, 1, 1)
elif mode == 'random_same_for_each_class':
    fixed_z = torch.randn   (nb_rows, 1,        nz, 1, 1)
    fixed_z = fixed_z.repeat(1,      nb_cols, 1, 1, 1)
    fixed_z = fixed_z.view(nb_rows * nb_cols, nz, 1, 1)
elif mode == 'interp_same_for_each_class':
    vals = torch.linspace(-3, 3, nb_cols)
    fixed_z = torch.zeros(nb_rows, nb_cols, nz, 1, 1)
    for i in range(nb_rows):
        fixed_z[i, :, i, 0, 0] = vals
    fixed_z = fixed_z.view(nb_rows * nb_cols, nz, 1, 1)
    print(fixed_z.size())
    
fixed_onehot = torch.zeros(nb_rows, nb_cols, nb_classes, 1, 1)
fixed_onehot = fixed_onehot.view(nb_rows * nb_cols, nb_classes, 1, 1)

for i in range(fixed_onehot.size(0)):
    cl = i % nb_classes
    if mode == 'interp_same_for_each_class':
        cl = CL
    fixed_onehot[i, cl] = 1
fixed_noise = torch.cat((fixed_z, fixed_onehot), 1)
#fixed_noise = Variable(fixed_z)
fixed_noise = Variable(fixed_noise)

In [ ]:
fake = G(fixed_noise)
#fake = fake.cuda()
#pr = nn.Softmax()(clf(norm(fake)))
#obj = compute_objectness(pr)
#print(obj)
#pr_max = pr.data.max(1)[1][:, 0].cpu()
#fo = fixed_onehot.max(1)[1][:, 0, 0, 0]
#print(torch.mean((pr_max==fo).float()))
im = (fake.data + 1) / 2.
vutils.save_image(im, 'samples.png', nrow=nb_cols)

In [ ]:
Image('samples.png', width=500, height=500)

In [ ]:
from ipywidgets import FloatSlider, IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual
from collections import OrderedDict

from IPython.display import display,clear_output
from ipywidgets import widgets
from collections import OrderedDict

%matplotlib inline
import matplotlib.pyplot as plt

def update(w):
    clear_output()
    if 'cl' in w:
        cl = int(w['cl'].value)
    else:
        cl = None
    z = [w[str(i)].value for i in range(nz)]
    z = np.array(z)
    z = z[np.newaxis, :, np.newaxis, np.newaxis]
    z = z.astype(np.float32)
    z = torch.from_numpy(z)
    z = z.repeat(32, 1, 1, 1)
    if cl is not None:
        print(cl)
        onehot = torch.zeros(z.size(0), nb_classes)
        onehot[:, cl] = 1
        z = torch.cat((z, onehot), 1)
    z = Variable(z)
    fake = G(z)
    im = (fake.data + 1) / 2.
    im = im.cpu().numpy()
    im = im[0]
    im = im.transpose((1, 2, 0))
    plt.imshow(im)

w = OrderedDict()
for i in range(nz):
    w[str(i)] = FloatSlider(min=-4, max=4, step=0.1, value=0)
w['cl'] = IntSlider(min=0, max=nb_classes - 1, step=1, value=0)

for widget in w.values():
    widget.observe(lambda change:update(w),names='value',type='change')

vbox = widgets.VBox(children=list(w.values()))
display(vbox)
#interact(slow_function, **widgets)